In [38]:
import csv

filename = 'images.csv'

with open(filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['ID', 'Image URL']) 

In [37]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains
import csv
import string
import time

# Setting up the web driver
chrome_driver_path = "/Users/shivangidhiman/Downloads/chromedriver_mac64/chromedriver" 
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service)

url = "https://public.doc.state.sc.us/scdc-public/"  
driver.get(url)

for alphabet in string.ascii_lowercase:
    text_box = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "lastName"))) 
    text_box.clear()
    text_box.send_keys(alphabet)
    text_box.submit()

    # wait till the results are available
    wait = WebDriverWait(driver, 10)  
    is_results = False
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "svg.h-8.ml-0.cursor-pointer[style*='visibility: visible;']")))
        # Check if there are any results or not
        results = driver.find_element(By.CSS_SELECTOR, ".md\\:flex.flex-col.hidden.md\\:select-auto")
        if results.is_displayed():
            is_results = True
    except Exception:
        pass

    while(is_results):
        # scrapping data from the page
        new_page_html = driver.page_source
        soup = BeautifulSoup(new_page_html, "html.parser")
        image_tags = soup.select('div.flex-shrink-0.h-10.w-10 img')
        id_tags = soup.find_all(class_ = "px-2 inline-flex text-xs leading-5 font-semibold rounded-full bg-blue-100 text-blue-900")
        image_urls = []
        ids = []

        for img, id in zip(image_tags, id_tags):
            image_urls.append(img["src"])
            ids.append(id.text)

        # Saving info in csv
        filename = 'images.csv'

        data = list(zip(ids, image_urls))

        # Filter out data with blank elements
        data = [t for t in data if all(element != "" for element in t)]

        with open(filename, 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(data)

        nextpage = driver.find_elements(By.CSS_SELECTOR, 'svg.h-8.ml-2.cursor-pointer[style*="visibility: visible;"]')
        if not nextpage:
            break

        actions = ActionChains(driver)
        actions.move_to_element(nextpage[0]).perform()
        time.sleep(0.5) 
        actions.click().perform()
        
driver.quit()   